In [28]:
import tensorflow as tf
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [29]:
Blinks = np.load('/content/drive/MyDrive/DLA/npy/Blinks_F3.npy')
Labels = np.load('/content/drive/MyDrive/DLA/npy/Labels_F3.npy')
BlinksTest = np.load('/content/drive/MyDrive/DLA/npy/BlinksTest_F3.npy')
LabelsTest = np.load('/content/drive/MyDrive/DLA/npy/LabelsTest_F3.npy')
    #deciding the indices of each video based on the fold
    #####################Normalizing the input#############Second phase
BlinksTest[:,:,0]=(BlinksTest[:,:,0]-np.mean(Blinks[:,:,0]))/np.std(Blinks[:,:,0])
Blinks[:,:,0]=(Blinks[:,:,0]-np.mean(Blinks[:,:,0]))/np.std(Blinks[:,:,0])
    #####
    #####
BlinksTest[:,:,1]=(BlinksTest[:,:,1]-np.mean(Blinks[:,:,1]))/np.std(Blinks[:,:,1])
Blinks[:,:,1]=(Blinks[:,:,1]-np.mean(Blinks[:,:,1]))/np.std(Blinks[:,:,1])
    #####
BlinksTest[:,:,2]=(BlinksTest[:,:,2]-np.mean(Blinks[:,:,2]))/np.std(Blinks[:,:,2])
Blinks[:,:,2]=(Blinks[:,:,2]-np.mean(Blinks[:,:,2]))/np.std(Blinks[:,:,2])
    #####
BlinksTest[:,:,3]=(BlinksTest[:,:,3]-np.mean(Blinks[:,:,3]))/np.std(Blinks[:,:,3])
Blinks[:,:,3]=(Blinks[:,:,3]-np.mean(Blinks[:,:,3]))/np.std(Blinks[:,:,3])
    ####
    ####JUST TO DOUBLE CHECK
    ####
    # print(np.mean(Blinks[:,:,0]),np.mean(Blinks[:,:,1]),np.mean(Blinks[:,:,2]),np.mean(Blinks[:,:,3]))
    # print(np.std(Blinks[:,:,0]),np.std(Blinks[:,:,1]),np.std(Blinks[:,:,2]),np.std(Blinks[:,:,3]))
    # print(np.mean(BlinksTest[:,:,0]),np.mean(BlinksTest[:,:,1]),np.mean(BlinksTest[:,:,2]),np.mean(BlinksTest[:,:,3]))
    # print(np.std(BlinksTest[:,:,0]),np.std(BlinksTest[:,:,1]),np.std(BlinksTest[:,:,2]),np.std(BlinksTest[:,:,3]))

In [48]:
#BinarizationLabels
LabelsTest[LabelsTest==10] = 1
Labels[Labels==10] = 1

In [82]:
def create_model(model_name):
    if model_name == 'LSTMreza':
      input = tf.keras.Input(shape=(30, 4))
      x = tf.keras.layers.Dense(64, activation='relu')(input)
      x = tf.keras.layers.Masking(mask_value=0.0)(x)
      x = tf.keras.layers.LSTM(32, return_sequences=False,dropout=0.1, recurrent_dropout=0.1)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dense(8, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.1)(x)
      y = tf.keras.layers.Dense(1, activation='tanh')(x)
      model = tf.keras.models.Model(inputs=[input], outputs=[y])
    elif model_name == 'LSTM':
      input = tf.keras.Input(shape=(30, 4))
      x = tf.keras.layers.Masking(mask_value=0.0)(input)
      x = tf.keras.layers.LSTM(32, return_sequences=False,dropout=0.1, recurrent_dropout=0.1)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.5)(x)
      y = tf.keras.layers.Dense(1, activation='tanh')(x)
      model = tf.keras.models.Model(inputs=[input], outputs=[y])
    elif model_name == 'stackedreza':
      input = tf.keras.Input(shape=(30, 4))
      x = tf.keras.layers.Dense(64, activation='relu')(input)
      x = tf.keras.layers.Masking(mask_value=0.0)(x)
      rnn_cells = [tf.keras.layers.LSTMCell(32, dropout=0.1, recurrent_dropout=0.1) for _ in range(4)]
      stacked_lstm = tf.keras.layers.StackedRNNCells(rnn_cells)
      x = tf.keras.layers.RNN(stacked_lstm)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.2)(x)
      x = tf.keras.layers.Dense(16, activation='relu')(x)
      x = tf.keras.layers.Dense(8, activation='relu')(x)
      x = tf.keras.layers.Dropout(0.1)(x)
      y = tf.keras.layers.Dense(1, activation='tanh')(x)
      model = tf.keras.models.Model(inputs=[input], outputs=[y])
    else:
      print("Chiama le cose con il loro nome")
    return model

In [99]:
model_name = "stackedreza"
model = create_model(model_name)

In [100]:
model.summary()

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 30, 4)]           0         
_________________________________________________________________
dense_60 (Dense)             (None, 30, 64)            320       
_________________________________________________________________
masking_19 (Masking)         (None, 30, 64)            0         
_________________________________________________________________
rnn_13 (RNN)                 (None, 32)                37376     
_________________________________________________________________
dense_61 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_20 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_62 (Dense)             (None, 16)                272

In [101]:
lr = 0.00053
bs = 128
ep = 80
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy']
)

In [ ]:
model.fit(x= Blinks, y = Labels, validation_data=(BlinksTest, LabelsTest), epochs=ep, batch_size=bs)

Epoch 1/80
61/61 [==============================] - 24s 224ms/step - loss: 2.1184 - accuracy: 0.4570 - val_loss: 1.1235 - val_accuracy: 0.2994
Epoch 2/80
61/61 [==============================] - 13s 209ms/step - loss: 0.9760 - accuracy: 0.4616 - val_loss: 0.8063 - val_accuracy: 0.2994
Epoch 3/80
61/61 [==============================] - 13s 210ms/step - loss: 0.8395 - accuracy: 0.5074 - val_loss: 0.6798 - val_accuracy: 0.6930
Epoch 4/80
61/61 [==============================] - 13s 208ms/step - loss: 0.7652 - accuracy: 0.5133 - val_loss: 0.6346 - val_accuracy: 0.7321
Epoch 5/80
61/61 [==============================] - 13s 210ms/step - loss: 0.7590 - accuracy: 0.5133 - val_loss: 0.6156 - val_accuracy: 0.7055
Epoch 6/80
61/61 [==============================] - 13s 210ms/step - loss: 0.7358 - accuracy: 0.5319 - val_loss: 0.6006 - val_accuracy: 0.7143
Epoch 7/80
61/61 [==============================] - 13s 210ms/step - loss: 0.7267 - accuracy: 0.5418 - val_loss: 0.5801 - val_accuracy: 0.7210

In [98]:
del model

In [ ]:
tf.compat.v1.disable_eager_execution()
training = tf.compat.v1.placeholder(tf.bool,name='phase_train')